In [ ]:
# Provjera dostupnog GPU-a
!nvidia-smi

Thu Feb  5 14:52:46 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!apt update
!apt install -y nvidia-cuda-toolkit

Baseline version profiling

In [ ]:
!nvcc -O3 -arch=sm_75 sort.cu -o sort

In [ ]:
!./sort

GPU sort time: 589.430 ms
Sorted!
Thrust RADIX sort time: 11.395 ms
Thrust MERGE sort time: 11.276 ms
========== SUMMARY ==========
Bitonic: 589.430 ms
Radix  : 11.395 ms
Merge  : 11.276 ms


In [ ]:
!nvcc -O3 -arch=sm_75 sort_bez.cu -o sort_bez

In [ ]:
!ncu --set full --target-processes all -o sort_bez ./sort_bez

==PROF== Connected to process 832 (/content/sort_bez)
==PROF== Profiling "bitonicSharedKernel(int *, int)" - 0: 0%.
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
...50%....100% - 30 passes
==PROF== Profiling "bitonicGlobalMerge" - 1: 0%....50%....100% - 30 passes
==PROF== Profiling "bitonicGlobalMerge" - 2: 0%....50%....100% - 30 passes
==PROF== Profiling "bitonicGlobalMerge" - 3: 0%....50%....100% - 30 passes
==PROF== Profiling "bitonicGlobalMerge" - 4: 0%....50%....100% - 30 passes
==PROF== Profiling "bitonicGlobalMerge" - 5: 0%....50%....100% - 30 passes
==PROF== Profiling "bitonicGlobalMerge" - 6: 0%....50%....100% - 30 passes
==PROF== Profiling "bitonicGlobalMerge" - 7: 0%....50%....100% - 30 passes
==PROF== Profiling "bitonicGloba

In [ ]:
!ncu --import sort_bez.ncu-rep --print-summary per-kernel

[832] sort_bez@127.0.0.1
  bitonicGlobalMerge(int *, int, int, int) (65536, 1, 1)x(1024, 1, 1), Device 0, CC 7.5, Invocations 306
    Section: Compute Workload Analysis
    -------------------- ----------- ------- ------- -------
    Metric Name          Metric Unit Minimum Maximum Average
    -------------------- ----------- ------- ------- -------
    Executed Ipc Active   inst/cycle    0.69    1.08    0.91
    Executed Ipc Elapsed  inst/cycle    0.58    0.88    0.73
    Issue Slots Busy               %   17.23   27.07   22.65
    Issued Ipc Active     inst/cycle    0.69    1.08    0.91
    SM Busy                        %   17.23   27.07   22.65
    -------------------- ----------- ------- ------- -------

    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------ ------------ ------------
    Metric Name             Metric Unit      Minimum      Maximum      Average
    ----------------------- ----------- ------------ ------------ ------------
 

Optimized version profiling

In [ ]:
!nvcc -O3 -arch=sm_75 perf_final.cu -o perf_final

In [ ]:
!./perf_final

Padding from 50000000 to 67108864
GPU bitonic sort time: 345.367 ms
Sorted correctly? YES


In [ ]:
!ncu --set full --target-processes all -o sum ./perf_final

In [ ]:
!ncu --import sum.ncu-rep --print-summary per-kernel

[26306] perf_final@127.0.0.1
  bitonicGlobalMergeOptimized(int *, int, int, int) (32768, 1, 1)x(1024, 1, 1), Device 0, CC 7.5, Invocations 136
    Section: Compute Workload Analysis
    -------------------- ----------- ------- ------- -------
    Metric Name          Metric Unit Minimum Maximum Average
    -------------------- ----------- ------- ------- -------
    Executed Ipc Active   inst/cycle    0.61    0.74    0.69
    Executed Ipc Elapsed  inst/cycle    0.54    0.63    0.60
    Issue Slots Busy               %   15.26   18.50   17.17
    Issued Ipc Active     inst/cycle    0.61    0.74    0.69
    SM Busy                        %   15.26   18.50   17.17
    -------------------- ----------- ------- ------- -------

    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ---------- ------------ ----------
    Metric Name             Metric Unit    Minimum      Maximum    Average
    ----------------------- ----------- ---------- ------------ ----------
